# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [309]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from scipy import stats
%matplotlib inline

## Read in an Image

In [310]:
#reading in an image
image_sWR = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image_sWR), 'with dimensions:', image_sWR.shape)
#plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)


## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [311]:
import math
from collections import deque


def find_hough_lines(img,lines):
    xl = []
    yl = []
    xr = []
    yr = []
    middel_x = img.shape[1]/2
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            if ((y2-y1)/(x2-x1))<0 and ((y2-y1)/(x2-x1))>-math.inf and x1<middel_x and x2<middel_x:
                xl.append(x1)
                xl.append(x2)
                yl.append(y1)
                yl.append(y2)

            elif ((y2-y1)/(x2-x1))>0 and ((y2-y1)/(x2-x1))<math.inf and x1>middel_x and x2>middel_x:
                xr.append(x1)
                xr.append(x2)
                yr.append(y1)
                yr.append(y2)
    
    return xl, yl, xr, yr

def linear_regression_left(xl,yl):
    slope_l, intercept_l, r_value_l, p_value_l, std_err = stats.linregress(xl, yl)
    return slope_l, intercept_l

def linear_regression_right(xr,yr):
    slope_r, intercept_r, r_value_r, p_value_r, std_err = stats.linregress(xr, yr)
    return slope_r, intercept_r

def create_lane_list():
    return deque(maxlen = 15)

def left_lane_mean(left_lane_que):
    if len(left_lane_que) == 0:
        return 0,0
    slope_l_mean , intercept_l_mean = np.mean(left_lane_que,axis=0)
    return slope_l_mean, intercept_l_mean

def right_lane_mean(right_lane_que):
    if len(right_lane_que) == 0:
        return 0,0
    slope_r_mean , intercept_r_mean = np.mean(right_lane_que,axis=0)
    return slope_r_mean, intercept_r_mean

def left_lane_add(left_lane_que,slope_l, intercept_l):
    left_lane_que.append([slope_l,intercept_l])
    return left_lane_que

def right_lane_add(right_lane_que,slope_r, intercept_r):
    right_lane_que.append([slope_r,intercept_r])
    return right_lane_que
        

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img):
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    vertices = get_vertices_for_img(img)  
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image           

def draw_lines(img, intercept_l, slope_l,intercept_r, slope_r, xl, xr,color=[255, 0, 0], thickness=10):
    
    max_y = img.shape[0]
    yl_LR = []
    yr_LR = []
    for x in xl:
        yl_LR.append(intercept_l+slope_l*x)
    for x in xr:
        yr_LR.append(intercept_r+slope_r*x)
        
    x_left_bottom = (max_y - intercept_l)/slope_l
    x_right_bottom = (max_y - intercept_r)/slope_r
    
    cv2.line(img, (int(x_left_bottom), int(max_y)), (int(max(xl)), int(min(yl_LR))), color, thickness)
    cv2.line(img, (int(x_right_bottom), int(max_y)), (int(min(xr)), int(min(yr_LR))), color, thickness)
    
    return img
    
    
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    return line_img, lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

def isolate_yellow_hsl(img):
    # Caution - OpenCV encodes the data in ****HLS*** format
    # Lower value equivalent pure HSL is (30, 45, 15)
    low_threshold = np.array([15, 38, 115], dtype=np.uint8)
    # Higher value equivalent pure HSL is (75, 100, 80)
    high_threshold = np.array([35, 204, 255], dtype=np.uint8)  
    
    yellow_mask = cv2.inRange(img, low_threshold, high_threshold)
    
    return yellow_mask
                            

# Image should have already been converted to HSL color space
def isolate_white_hsl(img):
    # Caution - OpenCV encodes the data in ***HLS*** format
    # Lower value equivalent pure HSL is (30, 45, 15)
    low_threshold = np.array([0, 200, 0], dtype=np.uint8)
    # Higher value equivalent pure HSL is (360, 100, 100)
    high_threshold = np.array([180, 255, 255], dtype=np.uint8)  
    
    white_mask = cv2.inRange(img, low_threshold, high_threshold)
    
    return white_mask

def get_vertices_for_img(img):
    imshape = img.shape
    height = imshape[0]
    width = imshape[1]

    
    if (width, height) == (960, 540):
        region_bottom_left = (130 ,img.shape[0] - 1)
        region_top_left = (410, 330)
        region_top_right = (650, 350)
        region_bottom_right = (img.shape[1] - 30,img.shape[0] - 1)
        vert = np.array([[region_bottom_left , region_top_left, region_top_right, region_bottom_right]], dtype=np.int32)
    else:
        region_bottom_left = (200 , 680)
        region_top_left = (600, 450)
        region_top_right = (750, 450)
        region_bottom_right = (1100, 680)
        vert = np.array([[region_bottom_left , region_top_left, region_top_right, region_bottom_right]], dtype=np.int32)
    return vert        

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [312]:
import os
os.listdir("test_images/")

['solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidWhiteRight.jpg',
 'solidYellowCurve2.jpg',
 'solidYellowCurve.jpg',
 'solidWhiteCurve.jpg',
 'challage.jpg']

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [320]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

image_wCLS = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')
image_sYL = mpimg.imread('test_images/solidYellowLeft.jpg')
image_sYC2 = mpimg.imread('test_images/solidYellowCurve2.jpg')
image_sYC = mpimg.imread('test_images/solidYellowCurve.jpg')
image_sWC = mpimg.imread('test_images/solidWhiteCurve.jpg')
image_ch = mpimg.imread('test_images/challage.jpg')
def Lane_Detect(image):
    image_hsl = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    
    image_yellow = isolate_yellow_hsl(image_hsl)
    image_white = isolate_white_hsl(image_hsl)
    image_wy = cv2.bitwise_or(image_yellow,image_white)
    image_com = cv2.bitwise_and(image,image,mask=image_wy)
    image_gray = grayscale(image_com)
    # Smoothing the image
    kernal_size = 11
    blur_image = cv2.GaussianBlur(image_gray,(kernal_size,kernal_size),0)
    # Setup Canny
    low_threshold = 10
    high_threshold = 150
    edges_image = cv2.Canny(blur_image,low_threshold,high_threshold)
    # Define range of interest
    masked_image = region_of_interest(edges_image) 
    
    bland_image, houghLines= hough_lines(masked_image, 1, np.pi/180, 1, 5, 1)
    xl,yl,xr,yr = find_hough_lines(bland_image,houghLines)
    slope_l, intercept_l = linear_regression_left(xl,yl)
    slope_r, intercept_r = linear_regression_right(xr,yr)
    hough_image = draw_lines(bland_image, intercept_l, slope_l, intercept_r, slope_r, xl, xr)
    Final_image = weighted_img(hough_image,image)  
    return Final_image
  
Final_wCLS = Lane_Detect(image_wCLS)
plt.imsave('test_images_output/whiteCarLaneSwitch.png',Final_wCLS)
Final_sWR = Lane_Detect(image_sWR)
plt.imsave('test_images_output/solidWhiteRight.png',Final_sWR)
Final_sYL = Lane_Detect(image_sYL)
plt.imsave('test_images_output/solidYellowLeft.png',Final_sYL)
Final_sYC2 = Lane_Detect(image_sYC2)
plt.imsave('test_images_output/solidYellowCurve2.png',Final_sYC2)
Final_sYC = Lane_Detect(image_sYC)
plt.imsave('test_images_output/solidYellowCurve.png',Final_sYC)
Final_sWC = Lane_Detect(image_sWC)
plt.imsave('test_images_output/solidWhiteCurve.png',Final_sWC)
Final_ch = Lane_Detect(image_ch)
plt.imsave('test_images_output/challange.png',Final_ch)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [321]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [322]:
MAXIMUM_SLOPE_DIFF = 0.1
MAXIMUM_INTERCEPT_DIFF = 50.0


class LaneDetectWithMemo:
    def __init__(self):
        self.left_lane_que = create_lane_list()
        self.right_lane_que = create_lane_list()
        
    def LanePipe(self,image):
        
        image_hsl = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    
        image_yellow = isolate_yellow_hsl(image_hsl)
        image_white = isolate_white_hsl(image_hsl)
        image_wy = cv2.bitwise_or(image_yellow,image_white)
        image_com = cv2.bitwise_and(image,image,mask=image_wy)
        
        image_gray = grayscale(image_com)
        # Smoothing the image
        kernal_size = 11
        blur_image = cv2.GaussianBlur(image_gray,(kernal_size,kernal_size),0)
        # Setup Canny
        low_threshold = 10
        high_threshold = 150
        edges_image = cv2.Canny(blur_image,low_threshold,high_threshold)
        # Define range of interest
        masked_image = region_of_interest(edges_image) 
        
        bland_image, houghLines= hough_lines(masked_image, 1, np.pi/180, 1, 5, 1)
        xl,yl,xr,yr = find_hough_lines(bland_image,houghLines)
        slope_l, intercept_l = linear_regression_left(xl,yl)
        slope_r, intercept_r = linear_regression_right(xr,yr)
        if len(self.left_lane_que) == 0 and len(self.right_lane_que) == 0:
            self.left_lane_que = left_lane_add(self.left_lane_que, slope_l, intercept_l)
            self.right_lane_que = right_lane_add(self.right_lane_que, slope_r, intercept_r)
            slope_l_mean, intercept_l_mean = left_lane_mean(self.left_lane_que)
            slope_r_mean, intercept_r_mean = right_lane_mean(self.right_lane_que)
        else:
            slope_l_mean, intercept_l_mean = left_lane_mean(self.left_lane_que)
            slope_r_mean, intercept_r_mean = right_lane_mean(self.right_lane_que)
            slope_l_diff = abs(slope_l-slope_l_mean)
            intercept_l_diff = abs(intercept_l-intercept_l_mean)
            slope_r_diff = abs(slope_r-slope_r_mean)
            intercept_r_diff = abs(intercept_r-intercept_r_mean)
            if intercept_l_diff < MAXIMUM_INTERCEPT_DIFF and slope_l_diff < MAXIMUM_SLOPE_DIFF:
                self.left_lane_que = left_lane_add(self.left_lane_que, slope_l, intercept_l)
                slope_l_mean, intercept_l_mean = left_lane_mean(self.left_lane_que)
            if intercept_r_diff < MAXIMUM_INTERCEPT_DIFF and slope_r_diff < MAXIMUM_SLOPE_DIFF:
                self.right_lane_que = right_lane_add(self.right_lane_que, slope_r, intercept_r)
                slope_r_mean, intercept_r_mean = right_lane_mean(self.right_lane_que)
                
        
        hough_image = draw_lines(bland_image, intercept_l_mean, slope_l_mean,intercept_r_mean, slope_r_mean, xl, xr)
        Final_image = weighted_img(hough_image,image)
        
        return Final_image


Let's try the one with the solid white lane on the right first ...

In [323]:
LaneDetect_1 = LaneDetectWithMemo()
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(LaneDetect_1.LanePipe) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4




  0%|          | 0/222 [00:00<?, ?it/s]

  2%|▏         | 5/222 [00:00<00:04, 45.07it/s]

  5%|▍         | 11/222 [00:00<00:04, 47.03it/s]

  7%|▋         | 15/222 [00:00<00:04, 44.22it/s]

  8%|▊         | 18/222 [00:00<00:05, 38.48it/s]

 10%|▉         | 22/222 [00:00<00:05, 37.39it/s]

 12%|█▏        | 26/222 [00:00<00:06, 31.07it/s]

 13%|█▎        | 29/222 [00:00<00:06, 30.55it/s]

 15%|█▍        | 33/222 [00:00<00:05, 32.48it/s]

 17%|█▋        | 37/222 [00:01<00:05, 33.72it/s]

 18%|█▊        | 41/222 [00:01<00:05, 35.36it/s]

 20%|██        | 45/222 [00:01<00:05, 33.97it/s]

 22%|██▏       | 49/222 [00:01<00:05, 29.37it/s]

 24%|██▍       | 53/222 [00:01<00:06, 24.98it/s]

 25%|██▌       | 56/222 [00:01<00:07, 23.53it/s]

 27%|██▋       | 59/222 [00:02<00:08, 19.50it/s]

 28%|██▊       | 62/222 [00:02<00:09, 16.14it/s]

 29%|██▉       | 65/222 [00:02<00:08, 17.45it/s]

 31%|███       | 68/222 [00:02<00:07, 19.40it/s]

 32%|███▏      | 71/222 [00:02<00:07, 20.40it/s]

 33%|███

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 30 s, sys: 867 ms, total: 30.8 s
Wall time: 12.7 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [302]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [324]:
LaneDetect_2 = LaneDetectWithMemo()
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(LaneDetect_2.LanePipe)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4




  0%|          | 0/682 [00:00<?, ?it/s]

  1%|          | 4/682 [00:00<00:18, 36.72it/s]

  1%|▏         | 9/682 [00:00<00:17, 38.83it/s]

  2%|▏         | 13/682 [00:00<00:17, 38.81it/s]

  2%|▏         | 17/682 [00:00<00:17, 37.81it/s]

  3%|▎         | 21/682 [00:00<00:18, 36.62it/s]

  4%|▎         | 25/682 [00:00<00:17, 36.81it/s]

  4%|▍         | 29/682 [00:00<00:17, 36.74it/s]

  5%|▍         | 33/682 [00:00<00:20, 32.38it/s]

  5%|▌         | 37/682 [00:01<00:22, 29.17it/s]

  6%|▌         | 42/682 [00:01<00:20, 31.62it/s]

  7%|▋         | 46/682 [00:01<00:21, 30.02it/s]

  7%|▋         | 50/682 [00:01<00:23, 26.74it/s]

  8%|▊         | 53/682 [00:01<00:29, 21.34it/s]

  8%|▊         | 56/682 [00:01<00:28, 21.73it/s]

  9%|▊         | 59/682 [00:02<00:31, 19.81it/s]

  9%|▉         | 62/682 [00:02<00:38, 15.95it/s]

 10%|▉         | 65/682 [00:02<00:37, 16.47it/s]

 10%|▉         | 68/682 [00:02<00:35, 17.41it/s]

 10%|█         | 70/682 [00:02<00:40, 14.93it/s]

 11%|█   

 57%|█████▋    | 390/682 [00:23<00:22, 13.25it/s]

 57%|█████▋    | 392/682 [00:23<00:20, 14.42it/s]

 58%|█████▊    | 394/682 [00:23<00:20, 13.79it/s]

 58%|█████▊    | 397/682 [00:23<00:17, 16.44it/s]

 59%|█████▊    | 399/682 [00:23<00:22, 12.51it/s]

 59%|█████▉    | 402/682 [00:23<00:18, 14.93it/s]

 59%|█████▉    | 405/682 [00:24<00:16, 16.69it/s]

 60%|█████▉    | 408/682 [00:24<00:16, 16.52it/s]

 60%|██████    | 411/682 [00:24<00:14, 18.43it/s]

 61%|██████    | 414/682 [00:24<00:13, 19.27it/s]

 61%|██████    | 417/682 [00:24<00:15, 16.94it/s]

 61%|██████▏   | 419/682 [00:24<00:15, 16.78it/s]

 62%|██████▏   | 421/682 [00:24<00:15, 16.78it/s]

 62%|██████▏   | 423/682 [00:25<00:17, 15.23it/s]

 62%|██████▏   | 425/682 [00:25<00:15, 16.31it/s]

 63%|██████▎   | 427/682 [00:25<00:19, 13.36it/s]

 63%|██████▎   | 429/682 [00:25<00:17, 14.58it/s]

 63%|██████▎   | 432/682 [00:25<00:14, 17.22it/s]

 64%|██████▍   | 435/682 [00:25<00:17, 14.14it/s]

 64%|██████▍   | 437/682 [00:26

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 1min 43s, sys: 2.96 s, total: 1min 46s
Wall time: 41.3 s


In [308]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [325]:
LaneDetect_ch = LaneDetectWithMemo()
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(LaneDetect_ch.LanePipe)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4




  0%|          | 0/251 [00:00<?, ?it/s]

  1%|          | 3/251 [00:00<00:11, 22.01it/s]

  2%|▏         | 6/251 [00:00<00:10, 23.25it/s]

  4%|▎         | 9/251 [00:00<00:09, 24.26it/s]

  5%|▍         | 12/251 [00:00<00:09, 24.27it/s]

  6%|▌         | 14/251 [00:00<00:10, 22.62it/s]

  7%|▋         | 17/251 [00:00<00:09, 23.73it/s]

  8%|▊         | 20/251 [00:00<00:09, 24.31it/s]

  9%|▉         | 23/251 [00:00<00:09, 24.05it/s]

 10%|█         | 26/251 [00:01<00:09, 24.31it/s]

 12%|█▏        | 29/251 [00:01<00:09, 24.64it/s]

 13%|█▎        | 32/251 [00:01<00:08, 25.24it/s]

 14%|█▍        | 35/251 [00:01<00:08, 25.75it/s]

 15%|█▌        | 38/251 [00:01<00:08, 25.73it/s]

 16%|█▋        | 41/251 [00:01<00:08, 25.69it/s]

 18%|█▊        | 44/251 [00:01<00:08, 24.12it/s]

 19%|█▊        | 47/251 [00:02<00:11, 17.12it/s]

 20%|█▉        | 50/251 [00:02<00:12, 16.51it/s]

 21%|██        | 52/251 [00:02<00:13, 14.85it/s]

 22%|██▏       | 54/251 [00:03<00:26,  7.56it/s]

 22%|██▏  

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 57.7 s, sys: 1.9 s, total: 59.6 s
Wall time: 29 s


In [319]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))